# Apply association rule to find rules with the current dataset? For example, a customer who purchased peanut butter and jelly together has also purchased bread

In [1]:
# import required lib
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar 
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

# Load data from all the sources

In [47]:
#Read the data
orders_all = pd.read_csv("perf_test_orderdata/orders_all.csv")
orders_times = pd.read_csv("perf_test_orderdata/orders_times.csv")

In [48]:
#join the column to connect the two csv data
merge = pd.merge(orders_all,orders_times,how='left',on='admin_reference')

In [49]:
#load product data collected from API
products = pd.read_csv("products.csv")

# Prepare Data

In [50]:
#drop the features which has most null values
orders = merge.drop(['completed_at_x','customer_company','bill_state_name','ship_state_name','ship_company','subsite_store','campaign_code','bill_company'],axis=1)

In [51]:
#for campaign 
campaign_code = merge.drop(['completed_at_x','customer_company','bill_state_name','ship_state_name','ship_company','subsite_store','bill_company'],axis=1)

In [52]:
#prepasre the dat 
def prep_data(data):
    #Date and time is splitted
    new = data["completed_at_y"].str.split(" ", n = 1, expand = True) 
    data['Date'] = new[0]
    data['Time'] = new[1]
    #Day, month and year is splitted
    new = data["Date"].str.split("-", n = 2, expand = True) 
    data['Year'] = new[0]
    data['Month'] = new[1]
    data['Day'] = new[2]
    #drop null values
    data = data.dropna()
    #convert month number to month name
    data['Month'] = data['Month'].astype(int).apply(lambda x: calendar.month_abbr[x])
    #Date and time is splitted
    new = data["Time"].str.split(":", n = 1, expand = True) 
    data['Hour'] = new[0]
    data['Minute'] = new[1]

    data['Date'] = pd.to_datetime(data['Date'])  # Step 1
    data['DayofWeek'] =data['Date'].dt.day_name()  # Step 2
    return data

In [53]:
orders = prep_data(orders)
campaign_code = prep_data(campaign_code)

In [54]:
#update values in orders table
# orders['group_name'] = None
# orders.update(products)

In [55]:
orders = orders.drop([ 'state', 'shipment_state',
       'currency', 'bill_zipcode',
       'ship_city', 'ship_zipcode', 'ship_country_iso_name'],axis=1)

In [56]:
orders.head(2)

,admin_reference,payment_state,total,bill_city,bill_country_iso_name,product_name,quantity,sku,completed_at_y,Date,Time,Year,Month,Day,Hour,Minute,DayofWeek
1,O160651894,paid,97.21,Hafrsfjord,NO,AROMA Svartvinbärstoppar 900g,1,WEB7098,2018-05-31 09:08,2018-05-31,09:08,2018,May,31,09,08,Thursday
2,O160651894,paid,97.21,Hafrsfjord,NO,AROMA HALLON & LAKRITSBÅTAR 900G 2:a sor,1,WEB7080,2018-05-31 09:08,2018-05-31,09:08,2018,May,31,09,08,Thursday


In [57]:
products = products.drop(['ID', 'product_id', 'ean', 'is_master', 'weight',
       'reference', 'source_owner', 'source_id', 'best_price', 'stock_available_qty', 'sku_api',
       'group_description', 'group_properties','current_price'],axis=1)

In [58]:
products.head(2)

,sku,name,stock_qty,group_name
0,COC520,Monster Lewis Hamilton 50 cl x 24 st,54,Energidryck
1,SÄL427025,SMAKIS APELSIN KRAV BRICK 25CL - 27 st,18,web-shop-products
